# Equipo 36

| Nombre | Matrícula |
| ------ | --------- |
| André Martins Cordebello | A00572928 |
| Enrique Eduardo Solís Da Costa | A00572678 |
| Delbert Francisco Custodio Vargas | A01795613 |

# Reporte de problemas fitosanitarios en plantaciones de agave
--------------------

## Avance 1: análisis exploratorio de datos

Este primer avance consiste en realizar un análisis exploratorio de datos (EDA - Exploratory Data Analysis), es decir, describir los datos utilizando técnicas estadísticas y de visualización (análisis univariante y bi/multivariante) para hacer enfoque en sus aspectos más relevantes, así como aplicar y justificar operaciones de preprocesamiento, relacionadas con el manejo de valores faltantes, atípicos y alta cardinalidad. Es importante que incluyan sus conclusiones del EDA, identificando tendencias o relaciones importantes.

### Cargando los datos

La CNIT compartió 2 archivos de Excel en los cuáles se encuentra el muestreo de la presencia de gorgojos del agave detectados por sensores conocidos como "trampas". Dichas trampas se encuentran esparcidas por el estado de Jalisco, el cual cuenta con presencia de operaciones de la CNIT.

En dichos datasets encontramos los siguientes tipos de datos:

| Dato | Tipo | ¿Qué significa? |
| ---- | ---- | --------------  |
| ID Trampa	Fecha Trampeo | Text | |
| Latitud | Float | |
| Longitud  | Float | |
| Municipio | Texto | |
| Superficie (Has) | Float | |
| Edad de la plantación en años | Entero | | 
| No. de Capturas | Entero | |
| Cebo | Texto | |
| Estado | Texto | |
| PAIS  | Texto | |
| UBICACIÓN | Texto | |
| SEMA | Texto | |
| Mun  | Texto | |
| Est  | Texto | |



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os


plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [2]:
# Carga de datasets

historic_captures_dataframe     = pd.read_excel( "Capturas_Picudo_2014_a_2024.xlsx", sheet_name="Datos", header= 0)
current_year_captures_dataframe = pd.read_excel("Capturas_Picudo_2025.xlsx", sheet_name="DB", header=0)

### Comparación de columnas
----------------------------

Al comparar las columnas que tiene cada dataset incluído, es posible notar que la columna `Empresa` de los datos del año  2025 no fue tomada en cuenta para el DataSet histórico que incluye información del 2014 al 2024. Por lo tanto, debemos preguntar a Luis Felipe y Axel si ellos pueden explicarnos la causa de que en muestreos anteriores esta información no existe o no era importante.

In [3]:
print(historic_captures_dataframe.columns)
print(current_year_captures_dataframe.columns)

Index(['ID Trampa', 'Fecha Trampeo', 'Latitud ', 'Longitud', 'Municipio',
       'Superficie (Has)', 'Edad de la plantación en años', 'No. de Capturas',
       'Cebo', 'Estado', 'PAIS', 'UBICACIÓN', 'SEMA', 'Mun Est'],
      dtype='object')
Index(['ID Trampa', 'Fecha Trampeo', 'Latitud ', 'Longitud', 'Municipio',
       'Superficie (Has)', 'Edad de la plantación en años', 'No. de Capturas',
       'Cebo', 'Estado', 'PAIS', 'UBICACIÓN', 'SEMA', 'Mun Est', 'Empresa'],
      dtype='object')


Por lo tanto, prematuramente estaremos ignorando esta columna hasta conocer más sobre el dataset por medio del apoyo de nuestro sponsor.

In [4]:
current_year_captures_dataframe.drop( columns= ['Empresa'], index=1, inplace=True)

Luego nos preparamos para unir los DataFrames...

In [8]:
all_historic_captures_df = pd.concat( [historic_captures_dataframe, current_year_captures_dataframe], ignore_index=True )
print("Columnas disponibles: " + str(all_historic_captures_df.columns) + "\n\n===================")

all_historic_captures_df.dtypes

Columnas disponibles: Index(['ID Trampa', 'Fecha Trampeo', 'Latitud ', 'Longitud', 'Municipio',
       'Superficie (Has)', 'Edad de la plantación en años', 'No. de Capturas',
       'Cebo', 'Estado', 'PAIS', 'UBICACIÓN', 'SEMA', 'Mun Est'],
      dtype='object')



ID Trampa                                object
Fecha Trampeo                    datetime64[ns]
Latitud                                 float64
Longitud                                float64
Municipio                                object
Superficie (Has)                        float64
Edad de la plantación en años           float64
No. de Capturas                         float64
Cebo                                     object
Estado                                   object
PAIS                                     object
UBICACIÓN                                object
SEMA                                     object
Mun Est                                  object
dtype: object

In [9]:
all_historic_captures_df.describe(include=["number"]).T

,count,mean,std,min,25%,50%,75%,max
Latitud,828120.0,20.642468,0.728738,18.987685,20.161839,20.597546,20.884595,23.439659
Longitud,828120.0,-102.755746,1.301064,-114.108837,-103.720535,-102.650350,-102.184024,-98.662612
Superficie (Has),849346.0,5.320690,14.194881,0.000000,0.000000,0.800000,4.060000,787.820000
Edad de la plantación en años,849346.0,2.453746,1.827056,0.000000,1.000000,2.000000,4.000000,19.000000
No. de Capturas,849610.0,4.306977,8.188936,0.000000,0.000000,2.000000,5.000000,427.000000


In [10]:
all_historic_captures_df.describe(exclude=["number"]).T

,count,unique,top,freq,mean,min,25%,50%,75%,max
ID Trampa,849610.0,84466.0,716.0,125.0,NaN,NaN,NaN,NaN,NaN,NaN
Fecha Trampeo,849610,NaN,NaN,NaN,2019-02-20 13:57:23.510316032,2014-01-01 00:00:00,2016-05-31 00:00:00,2017-12-05 00:00:00,2022-06-13 00:00:00,2025-08-31 00:00:00
Municipio,849610,282,Arandas,60869,NaN,NaN,NaN,NaN,NaN,NaN
Cebo,837368,7,Cebo alimenticio,617509,NaN,NaN,NaN,NaN,NaN,NaN
Estado,849610,9,JALISCO,331660,NaN,NaN,NaN,NaN,NaN,NaN
PAIS,849610,1,MEX,849610,NaN,NaN,NaN,NaN,NaN,NaN
UBICACIÓN,849610,406,"Arandas ,JALISCO ,MEX",60464,NaN,NaN,NaN,NaN,NaN,NaN
SEMA,849610,5,1-25,535703,NaN,NaN,NaN,NaN,NaN,NaN
Mun Est,849610,406,"Arandas ,JALISCO",60464,NaN,NaN,NaN,NaN,NaN,NaN


### Porcentaje de valores faltantes por columna o característica

In [14]:
all_historic_captures_df.isna().sum() / all_historic_captures_df.count() * 100

ID Trampa                        0.000000
Fecha Trampeo                    0.000000
Latitud                          2.595035
Longitud                         2.595035
Municipio                        0.000000
Superficie (Has)                 0.031083
Edad de la plantación en años    0.031083
No. de Capturas                  0.000000
Cebo                             1.461962
Estado                           0.000000
PAIS                             0.000000
UBICACIÓN                        0.000000
SEMA                             0.000000
Mun Est                          0.000000
dtype: float64